In [ ]:
import nltk
import pandas as pd
import os
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')

In [ ]:
from collections import Counter
def flattenListOfLists(lst):
  result = []
  for sublist in lst:
      result.extend(sublist)
  return result

def compute_stats(df):
    lst = flattenListOfLists(df['source_tag'].tolist())
    s_counts = Counter( tag for word,  tag in lst)
    # print("source tags:", s_counts)
    lst = flattenListOfLists(df['edge_tag'].tolist())
    e_counts = Counter( tag for word,  tag in lst)
    # print("edge tags:", e_counts)
    lst = flattenListOfLists(df['target_tag'].tolist())
    t_counts = Counter( tag for word,  tag in lst)
    # print("target tags:", t_counts)
    return s_counts, t_counts, e_counts

def compare_vals(df1, df2, col):
    matching_count = 0
    for ind in (df1.loc[df1[col] == df2[col]].index):
        x =  df1.loc[df1.index == ind, col].values[0]
        y =  df2.loc[df2.index == ind, col].values[0]
        matching_count += 1
        
    return matching_count / len(df1)

In [ ]:
%cd ../textacy

In [ ]:
# %cd ~/../content/drive/MyDrive/lm_experiments
csv_names = [name for name in os.listdir() if 'kg_df.csv' in name]
print(csv_names)
tag_counts = {}
accuracy = {}
for i in range(0,len(csv_names),2):
  name = csv_names[i]
  df = pd.read_csv(name, index_col=0)
  df['source_tag'] = df['source'].apply(nltk.word_tokenize).apply(nltk.pos_tag, tagset="universal")
  df['target_tag'] = df['target'].apply(nltk.word_tokenize).apply(nltk.pos_tag, tagset="universal")
  df['edge_tag'] = df['edge'].apply(nltk.word_tokenize).apply(nltk.pos_tag, tagset="universal")
  s_counts, t_counts, e_counts = compute_stats(df)
  total_stats = s_counts + t_counts + e_counts
  print(total_stats)
  name_g = csv_names[i+1]
  df2 = pd.read_csv(name_g, index_col=0)
  df2['source_tag'] = df2['source'].apply(nltk.word_tokenize).apply(nltk.pos_tag, tagset="universal")
  df2['target_tag'] = df2['target'].apply(nltk.word_tokenize).apply(nltk.pos_tag, tagset="universal")
  df2['edge_tag'] = df2['edge'].apply(nltk.word_tokenize).apply(nltk.pos_tag, tagset="universal")
  print(name_g, name)
  s2_counts, t2_counts, e2_counts = compute_stats(df2)
  total_g_stats = s2_counts + t2_counts + e2_counts    
  print(total_g_stats, total_g_stats.values())

  diff = Counter({key: (round((value - (total_g_stats.get(key, 0)))*100/(total_g_stats.get(key, 0)),3) if total_g_stats.get(key, 0) != 0 else 0)  for key, value in total_stats.items()})
  print(diff)
  tag_counts[name] = diff

import pandas as pd

(pd.DataFrame.from_dict(data=tag_counts, orient='index')
   .to_csv('percent_tag_counts.csv', header=False))

In [ ]:
tag_counts

In [ ]:
df = pd.DataFrame.from_dict(tag_counts, orient='index').stack().reset_index()
df.columns = ['file', 'POS', 'count']
# df = df.set_index(['file'])
df = df.pivot('file', 'POS','count')
df.to_csv('percent_tag_counts.csv')

In [ ]:
def compare(D1, D2):
  return Counter({key: D2.get(key, 0) - value for key, value in D1.items()})

compare(tag_counts['squad_Bert_g_kg_df.csv'][0], tag_counts['squad_Bert_kg_df.csv'][0])

In [ ]:
compare(tag_counts['squad_Bert_g_kg_df.csv'][1], tag_counts['squad_Bert_kg_df.csv'][1])

In [ ]:
model = 'Bert'
dataset = 're-date-birth'
# 0 for source, 1 for target, 2 for edge
part_of_model = 0

name = dataset + "_" + model + '_kg_df.csv'
g_name = dataset + "_" + model + '_g_kg_df.csv'
compare(tag_counts[g_name][part_of_model], tag_counts[name][part_of_model])